# Для АТС препаратов, относящихся к лечению диабета, будем парсить страницу с описанием препаратов


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Подключаем WebDriverWait для ожидания загрузки элементов страницы
# expected_conditions (EC) содержит условия, при которых Selenium ждет появления элементов

📌 Комментарий:

WebDriverWait используется для ожидания появления элементов на странице перед взаимодействием с ними.
expected_conditions (EC) содержит предопределенные условия, такие как наличие элемента, его кликабельность и другие, которые помогают Selenium работать с динамическим контентом.
Эти модули часто используются в парсинге сайтов с динамически загружаемым контентом (например, при загрузке элементов через JavaScript).

In [ ]:
import pandas as pd  # Импортируем библиотеку pandas для работы с табличными данными

# Загружаем CSV-файл с данными об ATC-категориях для диабетических препаратов
df = pd.read_csv('data/ATC_categories_diabetis.csv', index_col=0)

# Извлекаем информацию о лекарственных средствах (их названия) и ATC-коды
drugs = df['ATC_info'].values  # Получаем список названий препаратов
ATCs = df['ATC'].values  # Получаем список ATC-кодов

# Выводим количество загруженных препаратов
print(len(drugs))

# Выводим первые три названия препаратов и их ATC-коды
print(drugs[:3], ATCs[:3])


173
['Calcium glucoheptonate' 'Teplizumab' 'Tolrestat'] ['A12AA10' 'A10XX01' 'A10XA01']


📌 Комментарий:

- import pandas as pd — загружаем библиотеку pandas, которая используется для обработки табличных данных.
- pd.read_csv('data/ATC_categories_diabetis.csv', index_col=0) — загружаем CSV-файл, где:
- index_col=0 указывает, что первый столбец должен использоваться как индекс.
- df['ATC_info'].values — извлекает массив с названиями лекарственных препаратов.
-зdf['ATC'].values — извлекает массив с ATC-кодами (классификация препаратов).
- print(len(drugs)) — выводит общее количество загруженных записей.
- print(drugs[:3], ATCs[:3]) — показывает первые три названия препаратов и их ATC-коды для проверки корректности загрузки данных.

In [ ]:
import undetected_chromedriver as uc  # Импортируем undetected_chromedriver для обхода антибот-защиты
from selenium.webdriver.common.by import By  # Импортируем By для поиска элементов на странице
import time  # Подключаем библиотеку time для управления задержками
from selenium import webdriver  # Импортируем webdriver для управления браузером
from webdriver_manager.chrome import ChromeDriverManager  # Автоматическая установка ChromeDriver
from selenium.webdriver.common.keys import Keys  # Импортируем Keys для имитации нажатия клавиш

# Инициализируем Chrome через undetected_chromedriver, чтобы обходить антибот-защиту Cloudflare
options = uc.ChromeOptions()
options.add_argument('--no-sandbox')  # Отключаем песочницу для повышения совместимости
options.add_argument('--disable-gpu')  # Отключаем GPU (актуально для некоторых серверов)
options.add_argument('--disable-dev-shm-usage')  # Отключаем совместное использование памяти
driver = uc.Chrome(options=options)  # Запускаем браузер с заданными параметрами

# Цикл для обработки каждого лекарства из списка
for i, drug_name in enumerate(drugs):
    try:
        ATC = ATCs[i]  # Получаем ATC-код, соответствующий лекарству
        time.sleep(2)  # Добавляем небольшую задержку для стабильности

        url = "https://go.drugbank.com/"  # URL сайта DrugBank
        driver.get(url)  # Загружаем страницу
        time.sleep(3)  # Ждем, чтобы Cloudflare успел проверить соединение

        # Находим поле поиска по XPath и вводим название лекарства
        xpath = '/html/body/header/div/nav/div/form/div[2]/input'
        fld = driver.find_element(By.XPATH, xpath)  # Ищем поле ввода
        fld.click()  # Активируем поле ввода
        fld.send_keys(drug_name)  # Вводим название лекарства
        fld.send_keys(Keys.RETURN)  # Нажимаем Enter для поиска

        time.sleep(3)  # Ждем загрузки страницы с результатами поиска

        # Получаем HTML-код страницы
        page1 = driver.page_source
        with open(f'drug_pages/{ATC}.html', 'w') as f:  # Сохраняем страницу в файл
            f.write(page1)

    except:
        continue  # Если возникает ошибка, пропускаем этот элемент и переходим к следующему


📌 Комментарий:

Импорт библиотек:

- undetected_chromedriver — используется для обхода защиты Cloudflare.
- selenium.webdriver.common.by.By — помогает искать элементы на странице.
- selenium.webdriver.common.keys.Keys — позволяет эмулировать нажатия клавиш.
- webdriver_manager.chrome — автоматически загружает ChromeDriver.

Настройка Chrome:

- options.add_argument('--no-sandbox') — отключает режим песочницы.
- options.add_argument('--disable-gpu') — отключает графику (актуально для серверов).
- options.add_argument('--disable-dev-shm-usage') — предотвращает ошибки с памятью.

Основной цикл обработки лекарств:

- Перебираем список drugs и соответствующие ATC-коды.
- Загружаем главную страницу DrugBank.
- Ожидаем проверку Cloudflare (time.sleep(3)).
- Используем find_element(By.XPATH, xpath), чтобы найти поле поиска и ввести название лекарства.
- Ждем загрузки результатов (time.sleep(3)).
- Сохраняем HTML-код страницы в файл в папке drug_pages/, используя ATC в названии.
Обработка ошибок:

- try-except предотвращает остановку программы при ошибке (например, если сайт временно недоступен).
- continue позволяет пропустить проблемный элемент и обработать следующий.